# Pre-processing Montana Water Right data for WaDEQA upload.

Date Updated: 08/05/2022

Purpose:  To pre-process the Montana data into one master file for simple DataFrame creation and extraction.

In [ ]:
#Needed Libararies

# working with data
import os
import numpy as np
import pandas as pd
import geopandas as gpd

# visulizaiton
import matplotlib.pyplot as plt
import seaborn as sns

# API retrieval
import requests
import json

# Cleanup
import re
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook
pd.set_option('display.float_format', lambda x: '%.5f' % x) # suppress scientific notation in Pandas

In [ ]:
# Working Directory
workingDir = "G:/Shared drives/WaDE Data/Montana/WaterAllocation/RawInputData"
os.chdir(workingDir)

## POD Water Budget Data

In [ ]:
# Input File
fileInput = "WaDE_PODs_input.zip"
dfinPOD = pd.read_csv(fileInput)

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in dfinPOD:
    dfinPOD['WaDEUUID'] = "mtD" + dfinPOD.index.astype(str)
    dfinPOD.to_csv('WaDE_PODs_input.zip', compression=dict(method='zip', archive_name='WaDE_PODs_input.csv'), index=False)

print(len(dfinPOD))
dfinPOD.head()

In [ ]:
# Clean Owner info.  Remove special characters
# need to separate out by ; into list
# remove special characters
# convert list to string, separate by ,

def cleanOwnerDataFunc(Val):
    Val = Val.strip()
    ValList = Val.split(';')
    for i, s in enumerate(ValList):
        ValList[i] = re.sub("[$@&.,;/\)(-]", "", s).title().strip()
    outString = ','.join(ValList)
    return outString

dfinPOD['ALL_OWNERS'] = dfinPOD.apply(lambda row: cleanOwnerDataFunc(row['ALL_OWNERS']), axis=1)
dfinPOD['ALL_OWNERS'].unique()

In [ ]:
def createNativeLandingURLMTFunct(xVal):
    # convert to list
    xList = xVal.split(' ')
    
    # add '20' value to 2nd & 3rd positoin
    try:
        xList[1] = "20" + xList[1]
        xList[2] = "20" + xList[2]
        outVal = ' '.join(xList).replace(' ', '%')
    except:
        outVal = ' '.join(xList).replace(' ', '%')
    
    # concatenate with url
    outstring = "http://wrqs.dnrc.mt.gov/ResultsWS.aspx?search=simple&index=8&wrnumber=" + outVal + "&status=ACTV!SEVR"
    
    return outstring

dfinPOD['in_WaterAllocationNativeURL'] = dfinPOD.apply(lambda row: createNativeLandingURLMTFunct(row['WR_NUMBER']), axis=1)
dfinPOD['in_WaterAllocationNativeURL'].unique()

In [ ]:
# create output POD dataframe
df = pd.DataFrame()

# Data Assessment UUID
df['WaDEUUID'] = dfinPOD['WaDEUUID']

# Method Info
df['in_MethodUUID'] = "MTwr_M1"

# Variable Info
df['in_VariableSpecificUUID'] = "MTwr_V1"

# Organization Info
df['in_OrganizationUUID'] = "MTwr_O1"

# WaterSource Info
df['in_Geometry'] = ""
df['in_GNISFeatureNameCV'] = ""
df['in_WaterQualityIndicatorCV'] = ""
df['in_WaterSourceName'] = dfinPOD['SOURCE_NAM'].str.title()
df['in_WaterSourceNativeID'] = "" #auto fill in below
df['in_WaterSourceTypeCV'] = dfinPOD['SOURCE_TYP'].str.title()

# Site Info
df['in_CoordinateAccuracy'] = "WaDE Unspecified"
df['in_CoordinateMethodCV'] = "WaDE Unspecified"
df['in_County'] = dfinPOD['LLDS_COUNT'].str.title()
df['in_EPSGCodeCV'] = 4326
df['in_Geometry'] = ""
df['in_GNISCodeCV'] = ""
df['in_HUC12'] = dfinPOD['HUC_12']
df['in_HUC8'] = ""
df['in_Latitude'] = dfinPOD['Lat']
df['in_Longitude'] = dfinPOD['Long']
df['in_NHDNetworkStatusCV'] = ""
df['in_NHDProductCV'] = ""
df['in_PODorPOUSite'] = "POD"
df['in_SiteName'] = dfinPOD['DITCH_NAME'].str.title()
df['in_SiteNativeID'] = "POD" + dfinPOD['PODV_ID_SE'].replace("", 0).fillna(0).astype(int).astype(str)
df['in_SitePoint'] = ""
df['in_SiteTypeCV'] = dfinPOD['MEANS_OF_D'].str.title()
df['in_StateCV'] = "MT"
df['in_USGSSiteID'] = ""

# AllocationAmount Info
df['in_AllocationApplicationDate'] = ""
df['in_AllocationAssociatedConsumptiveUseSiteIDs'] = ""
df['in_AllocationAssociatedWithdrawalSiteIDs'] = ""
df['in_AllocationBasisCV'] = ""
df['in_AllocationChangeApplicationIndicator'] = ""
df['in_AllocationCommunityWaterSupplySystem'] = ""
df['in_AllocationCropDutyAmount'] = ""
df['in_AllocationExpirationDate'] = ""
df['in_AllocationFlow_CFS'] = dfinPOD['FLW_RT_CFS'].astype(float)
df['in_AllocationLegalStatusCV'] = dfinPOD['WR_STATUS'].str.title()
df['in_AllocationNativeID'] =  dfinPOD['WR_NUMBER'].replace("", 0).fillna(0).astype(str)
df['in_AllocationOwner'] = dfinPOD['ALL_OWNERS']
df['in_AllocationPriorityDate'] = dfinPOD['ENF_PRIORI']
df['in_AllocationSDWISIdentifierCV'] = ""
df['in_AllocationTimeframeEnd'] = dfinPOD['PER_DIV_EN']
df['in_AllocationTimeframeStart'] = dfinPOD['PER_DIV_BG']
df['in_AllocationTypeCV'] = dfinPOD['WR_TYPE']
df['in_AllocationVolume_AF'] = dfinPOD['VOLUME']
df['in_BeneficialUseCategory'] = dfinPOD['PURPOSES'].str.replace("; ", ",").str.replace(", ", ",").str.title()
df['in_CommunityWaterSupplySystem'] = ""
df['in_CropTypeCV'] = ""
df['in_CustomerTypeCV'] = ""
df['in_DataPublicationDate'] = ""
df['in_DataPublicationDOI'] = ""
df['in_ExemptOfVolumeFlowPriority'] = 0
df['in_GeneratedPowerCapacityMW'] = ""
df['in_IrrigatedAcreage'] = dfinPOD['MAX_ACRES']
df['in_IrrigationMethodCV'] = ""
df['in_LegacyAllocationIDs'] = ""
df['in_OwnerClassificationCV'] = ""
df['in_PopulationServed'] = ""
df['in_PowerType'] = ""
df['in_PrimaryBeneficialUseCategory'] = ""
df['in_SDWISIdentifierCV'] = ""
df['in_WaterAllocationNativeURL'] = dfinPOD['in_WaterAllocationNativeURL']

outPOD = df.copy()
outPOD = outPOD.drop_duplicates().reset_index(drop=True)
print(len(outPOD))
outPOD.head()

## PoU Water Budget Data

In [ ]:
# Input File
fileInput = "WaDE_PoUs_input.zip"
dfinPOU = pd.read_csv(fileInput)

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in dfinPOU:
    dfinPOU['WaDEUUID'] = "mtU" + dfinPOU.index.astype(str)
    dfinPOU.to_csv('WaDE_PoUs_input.zip', compression=dict(method='zip', archive_name='WaDE_PoUs_input.csv'), index=False)

print(len(dfinPOU))
dfinPOU.head()

In [ ]:
def createNativeLandingURLMTFunct(xVal):
    # convert to list
    xList = xVal.split(' ')
    
    # add '20' value to 2nd & 3rd positoin
    try:
        xList[1] = "20" + xList[1]
        xList[2] = "20" + xList[2]
        outVal = ' '.join(xList).replace(' ', '%')
    except:
        outVal = ' '.join(xList).replace(' ', '%')
    
    # concatenate with url
    outstring = "http://wrqs.dnrc.mt.gov/ResultsWS.aspx?search=simple&index=8&wrnumber=" + outVal + "&status=ACTV!SEVR"
    
    return outstring

dfinPOU['in_WaterAllocationNativeURL'] = dfinPOU.apply(lambda row: createNativeLandingURLMTFunct(row['WRNUMBER']), axis=1)
dfinPOU['in_WaterAllocationNativeURL'].unique()

In [ ]:
# create output POD dataframe
df = pd.DataFrame()

# Data Assessment UUID
df['WaDEUUID'] = dfinPOU['WaDEUUID']

# Method Info
df['in_MethodUUID'] = "MTwr_M1"

# Variable Info
df['in_VariableSpecificUUID'] = "MTwr_V1"

# Organization Info
df['in_OrganizationUUID'] = "MTwr_O1"

# WaterSource Info
df['in_Geometry'] = ""
df['in_GNISFeatureNameCV'] = ""
df['in_WaterQualityIndicatorCV'] = ""
df['in_WaterSourceName'] = dfinPOU['SOURC_NAME'].str.title()
df['in_WaterSourceNativeID'] = "" #auto fill in below
df['in_WaterSourceTypeCV'] = dfinPOU['SRCTYPE']

# Site Info
df['in_CoordinateAccuracy'] = "WaDE Unspecified"
df['in_CoordinateMethodCV'] = "Centroid of Area"
df['in_County'] = dfinPOU['COUNTY']
df['in_EPSGCodeCV'] = 4326
df['in_Geometry'] = ""
df['in_GNISCodeCV'] = ""
df['in_HUC12'] = ""
df['in_HUC8'] = ""
df['in_Latitude'] = dfinPOU['Latitdue']
df['in_Longitude'] = dfinPOU['Longitdue']
df['in_NHDNetworkStatusCV'] = ""
df['in_NHDProductCV'] = ""
df['in_PODorPOUSite'] = "POU"
df['in_SiteName'] = "WaDE Unspecified"
df['in_SiteNativeID'] = "POU" + dfinPOU['OBJECTID'].replace("", 0).fillna(0).astype(int).astype(str)
df['in_SitePoint'] = ""
df['in_SiteTypeCV'] =  "WaDE Unspecified"
df['in_StateCV'] = "MT"
df['in_USGSSiteID'] = ""

# AllocationAmount Info
df['in_AllocationApplicationDate'] = ""
df['in_AllocationAssociatedConsumptiveUseSiteIDs'] = ""
df['in_AllocationAssociatedWithdrawalSiteIDs'] = ""
df['in_AllocationBasisCV'] = ""
df['in_AllocationChangeApplicationIndicator'] = ""
df['in_AllocationCommunityWaterSupplySystem'] = ""
df['in_AllocationCropDutyAmount'] = ""
df['in_AllocationExpirationDate'] = ""
df['in_AllocationFlow_CFS'] = dfinPOU['FLW_RT_CFS'].astype(float)
df['in_AllocationLegalStatusCV'] = dfinPOU['STATUS'].str.title()
df['in_AllocationNativeID'] =  dfinPOU['WRNUMBER'].replace("", 0).fillna(0).astype(str)
df['in_AllocationOwner'] = dfinPOU['ALL_OWNERS'].str.title()
df['in_AllocationPriorityDate'] = dfinPOU['ENF_PRIORI']
df['in_AllocationSDWISIdentifierCV'] = ""
df['in_AllocationTimeframeEnd'] = dfinPOU['PER_USE_EN']
df['in_AllocationTimeframeStart'] = dfinPOU['PER_USE_BG']
df['in_AllocationTypeCV'] = dfinPOU['WRTYPE'].str.title()
df['in_AllocationVolume_AF'] = dfinPOU['VOLUME']
df['in_BeneficialUseCategory'] = dfinPOU['PURPOSE'].str.replace("; ", ",").str.replace(", ", ",").str.title()
df['in_CommunityWaterSupplySystem'] = ""
df['in_CropTypeCV'] = ""
df['in_CustomerTypeCV'] = ""
df['in_DataPublicationDate'] = ""
df['in_DataPublicationDOI'] = ""
df['in_ExemptOfVolumeFlowPriority'] = 0
df['in_GeneratedPowerCapacityMW'] = ""
df['in_IrrigatedAcreage'] = dfinPOU['MAX_ACRES']
df['in_IrrigationMethodCV'] = ""
df['in_LegacyAllocationIDs'] = ""
df['in_OwnerClassificationCV'] = ""
df['in_PopulationServed'] = ""
df['in_PowerType'] = ""
df['in_PrimaryBeneficialUseCategory'] = ""
df['in_SDWISIdentifierCV'] = ""
df['in_WaterAllocationNativeURL'] = dfinPOU['in_WaterAllocationNativeURL']

outPOU = df.copy()
outPOU = outPOU.drop_duplicates().reset_index(drop=True)
print(len(outPOU))
outPOU.head()

## Concatenate POD and POU
## Fix Elements

In [ ]:
# Merge dataframes
frames = [outPOD, outPOU]
outdf = pd.concat(frames)
outdf = outdf.drop_duplicates().reset_index(drop=True)
print(len(outdf))

In [ ]:
# Fixing empty string names

def fixEmptyString(val):
    if val == "" or val == " " or val == "nan" or pd.isnull(val):
        outString = "WaDE Unspecified"
    else:
        outString = val
    return outString

In [ ]:
outdf['in_WaterSourceName'] = outdf.apply(lambda row: fixEmptyString(row['in_WaterSourceName']), axis=1)
outdf['in_WaterSourceName'].unique()

In [ ]:
outdf['in_SiteTypeCV'] = outdf.apply(lambda row: fixEmptyString(row['in_SiteTypeCV']), axis=1)
outdf['in_SiteTypeCV'].unique()

In [ ]:
outdf['in_AllocationTypeCV'] = outdf.apply(lambda row: fixEmptyString(row['in_AllocationTypeCV']), axis=1)
outdf['in_AllocationTypeCV'].unique()

In [ ]:
outdf['in_AllocationLegalStatusCV'] = outdf.apply(lambda row: fixEmptyString(row['in_AllocationLegalStatusCV']), axis=1)
outdf['in_AllocationLegalStatusCV'].unique()

In [ ]:
outdf['in_AllocationOwner'] = outdf.apply(lambda row: fixEmptyString(row['in_AllocationOwner']), axis=1)
outdf['in_AllocationOwner'].unique()

In [ ]:
outdf['in_BeneficialUseCategory'] = outdf.apply(lambda row: fixEmptyString(row['in_BeneficialUseCategory']), axis=1)
outdf['in_BeneficialUseCategory'].unique()

In [ ]:
# in_Latitude & in_Longitude
outdf['in_Latitude'] = pd.to_numeric(outdf['in_Latitude'], errors='coerce').fillna(0)
outdf['in_Longitude'] = pd.to_numeric(outdf['in_Longitude'], errors='coerce').fillna(0)
outdf.head(1)

In [ ]:
#Update datatype of Priority Date to fit WaDE 2.0 structure
outdf['in_AllocationPriorityDate'] = pd.to_datetime(outdf['in_AllocationPriorityDate'])
outdf['in_AllocationPriorityDate'] = pd.to_datetime(outdf['in_AllocationPriorityDate'].dt.strftime('%m/%d/%Y'))
outdf['in_AllocationPriorityDate'].unique()

In [ ]:
# Fixing in_AllocationFlow_CFS datatype
outdf['in_AllocationFlow_CFS'] = pd.to_numeric(outdf['in_AllocationFlow_CFS'], errors='coerce').fillna(0)
outdf['in_AllocationFlow_CFS'].unique()

In [ ]:
# Fixing in_AllocationVolume_AF datatype
outdf['in_AllocationVolume_AF'] = pd.to_numeric(outdf['in_AllocationVolume_AF'], errors='coerce').fillna(0)
outdf['in_AllocationVolume_AF'].unique()

In [ ]:
# Fixing in_IrrigatedAcreage datatype
outdf['in_IrrigatedAcreage'] = pd.to_numeric(outdf['in_IrrigatedAcreage'], errors='coerce').fillna(0)
outdf['in_IrrigatedAcreage'].unique()

In [ ]:
# fix WaterSourceTypeCV
# use WaDE specific terms only

waterSourceTypeDict = {
"SURFACE" : "Surface Water",
"GROUNDWATER" : "Groundwater",
"ALL NATURALLY OCCURING WATER" : "Surface Water"
}

def fixWaterSourceTypeCV(val):
    if val == "" or pd.isnull(val):
        outString = "WaDE Unspecified"
    else:
        String1 = val.strip()
        try:
            outString = waterSourceTypeDict[String1]
        except:
            outString = "WaDE Unspecified"
    return outString

outdf['in_WaterSourceTypeCV'] = outdf.apply(lambda row: fixWaterSourceTypeCV(row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceTypeCV'].unique()

In [ ]:
# Creating TimeframeStart & TimeframeEnd
# Spliting string, returning WaDE friendly format.

MonthNumbDict = {
"Jan" : "01",
"Feb" : "02",
"Mar" : "03",
"May" : "04",
"Apr" : "05",
"Jun" : "06",
"Jul" : "07",
"Aug" : "08",
"Sep" : "09",
"Oct" : "10",
"Nov" : "11",
"Dec" : "12"}

def createTimeframe(ColRowVal):
    val = str(ColRowVal)
    day = val.split('-')[0]
    try:
        month = val.split('-')[1]
        month = MonthNumbDict[month] + "/"
    except:
        day = ""
        month = ""
    outlist = month + day
    return outlist

outdf['in_AllocationTimeframeEnd'] = outdf.apply(lambda row: createTimeframe(row['in_AllocationTimeframeEnd']), axis=1)
outdf['in_AllocationTimeframeStart'] = outdf.apply(lambda row: createTimeframe(row['in_AllocationTimeframeStart']), axis=1)

outdf['in_AllocationTimeframeEnd'].unique()
outdf['in_AllocationTimeframeStart'].unique()

In [ ]:
# Creating WaDE Custom water source native ID for easy water source identification
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "wadeID" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceName'] = outdf['in_WaterSourceName']
dfWaterSourceNativeID['in_WaterSourceTypeCV'] = outdf['in_WaterSourceTypeCV']
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
def retrieveWaterSourceNativeID(A, B):
    if (A == '' and B == '') or (pd.isnull(A) and pd.isnull(B)):
        outList = ''
    else:
        ml = dfWaterSourceNativeID.loc[(dfWaterSourceNativeID['in_WaterSourceName'] == A) & 
                                       (dfWaterSourceNativeID['in_WaterSourceTypeCV'] == B), 'in_WaterSourceNativeID']
        if not (ml.empty):  # check if the series is empty
            outList = ml.iloc[0]
        else:
            outList = ''
    return outList

outdf['in_WaterSourceNativeID'] = outdf.apply(lambda row: retrieveWaterSourceNativeID( row['in_WaterSourceName'], row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceNativeID'].unique()

## Shapefile Data
- For attaching gemetry to POU csv inputs.

In [ ]:
# PoU Shapefile Data
# Shapefile input
dfPoUshapetemp = gpd.read_file('PoUShp/MT_PoU2.shp')
dfPoUshapetemp.head(3)

In [ ]:
columnsList = ['in_SiteNativeID', 'geometry']
dfPoUshape = pd.DataFrame(columns=columnsList)
dfPoUshape['in_SiteNativeID'] = "POU" + dfPoUshapetemp['OBJECTID'].replace("", 0).fillna(0).astype(int).astype(str)
dfPoUshape['geometry'] = dfPoUshapetemp['geometry']
dfPoUshape = dfPoUshape.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
dfPoUshape.head(3)

## Export Data

In [ ]:
outdf.info()

In [ ]:
outdf

In [ ]:
# Export the output dataframe
outdf.to_csv('Pwr_mtMain.zip', index=False, compression="zip")  # The output, save as a zip
dfPoUshape.to_csv('P_Geometry.zip', compression=dict(method='zip', archive_name='P_Geometry.csv'), index=False)  # The output geometry.